In [1]:
import cv2
import numpy as np
from scipy import ndimage as ndi
from skimage.morphology import square,dilation
from skimage.io import imshow,imread
import time
import matplotlib.pyplot as plt
import copy
import os
#DL
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Sequential,load_model
from keras.optimizers import Adam
import keras.utils
from keras.datasets import mnist
from keras import backend as K
%matplotlib inline

ImportError: cannot import name '_validate_lengths'

In [ ]:
x=[]
y=[]
path = 'C:/Users/Bermuda/Desktop/Hi!_Project4D/Project4D/Source Code/MLlane/image'
for fname in os.listdir(path):    
    paths=path+'/'+str(fname)
    for i in os.listdir(paths):
        pathss = paths+'/'+str(i)
        frame = imread(pathss)
        frame = cv2.resize(frame,(100, 70))
#         frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        y.append(fname)
        x.append(np.array(frame))  
        
x = np.array(x) 
y = np.array(y)  
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.20, random_state=53)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

In [ ]:
index = np.random.choice(x_train.shape[0], 21, replace=False)
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 4
for i in range(1, columns * rows + 1):
    img = x_train[index[i]]
    fig.add_subplot(rows, columns, i)
    plt.title(y_train[index[i]])
    plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
num_classes = 2

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255  # normalize dataset
# x_test /= 255
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train.shape)
print(y_train.shape)

In [ ]:

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(70, 100, 1))) #padding='same'ทำzero padding มาให้
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size=10
epochs=1
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save_weights('weight1.h5py')
model.save('my_model1.h5')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
kf = KFold(n_splits=10)
kf.get_n_splits(x_train)
for train_index, test_index in kf.split(x_train):
    X, X_ = x_train[train_index], x_train[test_index]
    Y, Y_ = y_train[train_index], y_train[test_index]
    model.fit(X, Y,batch_size=100,epochs=1)
#     print("Accuracysvc:",metrics.accuracy_score(Y_, y_pred))    
#     print(confusion_matrix(Y_, y_pred))  
#     print(classification_report(Y_, y_pred))
    

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])